In [2]:
import pickle as pkl
import datetime

import csv
# Scraping items
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

from IPython.core.display import display, HTML
import concurrent.futures

from collections import Counter
import numpy as np
from scipy import stats

import pandas as pd
from subprocess import call
import concurrent.futures
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#from pymongo import MongoClient

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
chromedriver = "/home/williamcottrell72/Downloads/chromedriver_linux64/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
# List of reviews
with open('SF_Urls','r') as f:
    urls=pd.read_csv(f)
urls2=[x.strip().strip('\'').strip('\'') for x in list(urls.columns)]

In [5]:
sf_attraction_template='https://www.tripadvisor.com/Attractions-g60713-Activities-oa{}-San_Francisco_California.html#FILTERED_LIST'


def make_url_list(sf_attraction_template):
    
    #The inital and other pages are slightly different so we just break these up
    
    initial_page=sf_attraction_template.format('')
    other_pages=[sf_attraction_template.format(30*i) for i in range(1,25)]
    all_pages=[initial_page]+other_pages
    
    urls_final=[]
    
    for p in all_pages:
        
        try:
            driver.get(p)
            soup=BeautifulSoup(driver.page_source,'html.parser')
            individual_urls=soup.find_all('div', class_='listing_title')

            for iu in individual_urls:
                partial_url=iu.find('a')['href']
                full_url='https://www.tripadvisor.com'+partial_url
                urls_final.append(full_url)
        except:
            pass
        
    return urls_final

In [6]:
#urls_final=make_url_list(sf_attraction_template)

# with open('urls_final.pkl','wb') as f:
#     pkl.dump(urls_final,f)

with open('urls_final.pkl','rb') as f:
    urls_final=pkl.load(f)

In [7]:
driver.get(urls_final[30])

In [16]:
location_info2={}
for url in urls_final:
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    try:
        rating=soup.find('span',class_='ui_bubble_rating bubble_40')['alt'][0]
    except:
        rating='None'
    try:
        tags=[x.text for x in soup.find('div',class_='detail').find_all('a',href=True)]
    except:
        tags=['None']
    name=soup.find(id='HEADING').text
    location_info2[name]=[rating,tags]

    


In [26]:
# with open('location_info.pkl','wb') as f:
#      pkl.dump(location_info,f)

with open('location_info2.pkl','rb') as f:
    location_info2 = pkl.load(f)

In [34]:
loc_info={x.strip('\n').strip('\u200e') : location_info2[x] for x in list(location_info2.keys())}

In [35]:
with open('data/loc_info.pkl','wb') as f:
    pkl.dump(loc_info,f)

In [ ]:
new_keys

In [19]:
Counter(tags+['Neighborhoods'])

Counter({'Sights & Landmarks': 1, 'Piers & Boardwalks': 1, 'Neighborhoods': 2})

In [ ]:
#Need map taking places to counts.
